# Evaluating the VLAAI network proposed in [Accurate decoding of the speech envelope using the VLAAI deep neural network](./#) on the DTU dataset.

In this example, a pre-trained VLAAI network will be evaluated on the publicly available dataset from [Fuglsang et al.](https://zenodo.org/record/1199011). This dataset contains 18 subjects who listened to one of two competing speech audio streams of 50 seconds with different levels of reverberation. For this example, we will only take the single-speaker trials (approximately 10 per subject, 500 seconds in total) into account.

The preprocessing used in this notebook is the same as proposed in the [paper](./#):
* For __EEG__: 
  1. High-pass filtering using a 1st order Butterworth filter with a cutoff frequency of 0.5Hz
  2. Downsampling to 1024 Hz
  3. Eyeblink artefact removal using a Multichannel Wiener filter
  4. Common average re-referencing
  5. Downsampling to 64Hz

* For __Speech__:
  1. Envelope extraction using a gamma-tone filterbank
  2. Downsampling to 1024 Hz
  3. Downsampling to 64 Hz

Preprocessed versions of the data are included in the [github repository](./#), code to the run the preprocessing manually is coming soon.

# Getting started

Installing the requirements

In [ ]:
# Clone the repository
!git clone https://github.com/berndie/vlaai
# Install the requirements
!pip3 install requirements.txt
!cd vlaai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 7.5 MB 3.2 MB/s 
     |████████████████████████████████| 1.7 MB 55.7 MB/s 
     |████████████████████████████████| 438 kB 47.5 MB/s 
     |████████████████████████████████| 5.9 MB 47.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

# Evaluating the pre-trained VLAAI network on the (already preprocessed) DTU dataset

In [ ]:
# General imports
import glob
import os.path
import numpy as np
import matplotlib.pyplot as plt

from model import vlaai
from examples.utils import window_data
from scipy.stats import pearsonr

In [ ]:
# Load the model
vlaai_model = vlaai()
vlaai_model.load_weights("pretrained_model/vlaai.h5")
vlaai_model.summary()

--2022-09-23 14:06:13--  https://zenodo.org/record/1199011/files/EEG.zip
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16044939963 (15G) [application/octet-stream]
Saving to: ‘EEG.zip.1’

EEG.zip.1           100%[===================>]  14.94G  18.3MB/s    in 17m 45s 

2022-09-23 14:24:00 (14.4 MB/s) - ‘EEG.zip.1’ saved [16044939963/16044939963]



In [ ]:
# Load the dataset
paths = glob.glob("evaluation_datasets/DTU/*.npz")
print("Found {} paths for evaluation".format(len(paths)))
subjects = set(["_".join(os.path.basename(x).split("_")[:2]) for x in paths])
print("Found {} subjects for evaluation".format(len(subjects)))

Archive:  EEG.zip
 extracting: S1.mat                  
 extracting: S10.mat                 
 extracting: S11.mat                 
 extracting: S12.mat                 
 extracting: S13.mat                 
 extracting: S14.mat                 
 extracting: S15.mat                 
 extracting: S16.mat                 
 extracting: S17.mat                 
 extracting: S18.mat                 
 extracting: S2.mat                  
 extracting: S3.mat                  
 extracting: S4.mat                  
 extracting: S5.mat                  
 extracting: S6.mat                  
 extracting: S7.mat                  
 extracting: S8.mat                  
 extracting: S9.mat                  


In [ ]:
# Run the model evaluation
subject_scores = {}
boxplot_data = []

# Iterate over the subjects in the DTU dataset
for subject in subjects:
    print("Evaluating subject {}".format(subject))
    for p in glob.glob("evaluation_datasets/DTU/{}_*.npz".format(subject)):
        print("Gathering scores for {}...".format(p))
        # Load the data
        # Data is stored in .npz format with two keys: 'eeg' and 'envelope'
        # containing preprocessed EEG and corresponding speech stimulus
        # envelope.
        data = np.load(p)
        eeg = data["eeg"]
        envelope = data["envelope"]

        # Standardize EEG and envelope
        eeg = (eeg - eeg.mean(axis=0, keepdims=True)) / eeg.std(
            axis=0, keepdims=True
        )
        envelope = (
            envelope - envelope.mean(axis=0, keepdims=True)
        ) / envelope.std(axis=0, keepdims=True)

        # Window the data in windows of 5 seconds with 80% overlap
        windowed_eeg = window_data(eeg, 320, 64)
        windowed_envelope = window_data(envelope, 320, 64)

        # Evaluate the model on the overlapping windows
        if subject not in subject_scores:
            subject_scores[subject] = []
        predictions = vlaai_model.predict(windowed_eeg)
        for pred, true in zip(predictions, windowed_envelope):
            r = pearsonr(pred.reshape(-1), true.reshape(-1))
            subject_scores[subject] += [r[0]]
    # Report the mean score for each subject
    mean_scores = np.mean(subject_scores[subject])
    boxplot_data += [mean_scores]
    print("Subject {}: {}".format(subject, mean_scores))

dict_keys(['__header__', '__version__', '__globals__', 'data', 'None', '__function_workspace__'])


In [ ]:
# Plot the results
plt.figure()
plt.boxplot(boxplot_data)
plt.ylabel("Pearson correlation")
plt.xlabel("Models")
plt.show()

KeyError: ignored

# COMING SOON: Code to do the preprocessing from scratch